# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3e604b1f40>
├── 'a' --> tensor([[-0.8319, -0.7362, -0.4087],
│                   [ 0.3903, -1.5619, -0.4698]])
└── 'x' --> <FastTreeValue 0x7f3e604b1e80>
    └── 'c' --> tensor([[-4.1987e-01,  1.0466e-03,  4.8218e-02, -1.9142e-01],
                        [-4.8661e-01, -1.4130e+00,  1.0928e+00, -1.6362e+00],
                        [ 1.5164e-01, -6.4227e-01,  2.5758e-01,  8.8379e-01]])

In [4]:
t.a

tensor([[-0.8319, -0.7362, -0.4087],
        [ 0.3903, -1.5619, -0.4698]])

In [5]:
%timeit t.a

68.6 ns ± 0.0932 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3e604b1f40>
├── 'a' --> tensor([[ 2.4518,  0.3391, -2.3361],
│                   [ 1.6329, -0.5250, -1.1297]])
└── 'x' --> <FastTreeValue 0x7f3e604b1e80>
    └── 'c' --> tensor([[-4.1987e-01,  1.0466e-03,  4.8218e-02, -1.9142e-01],
                        [-4.8661e-01, -1.4130e+00,  1.0928e+00, -1.6362e+00],
                        [ 1.5164e-01, -6.4227e-01,  2.5758e-01,  8.8379e-01]])

In [7]:
%timeit t.a = new_value

70.5 ns ± 0.0416 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8319, -0.7362, -0.4087],
               [ 0.3903, -1.5619, -0.4698]]),
    x: Batch(
           c: tensor([[-4.1987e-01,  1.0466e-03,  4.8218e-02, -1.9142e-01],
                      [-4.8661e-01, -1.4130e+00,  1.0928e+00, -1.6362e+00],
                      [ 1.5164e-01, -6.4227e-01,  2.5758e-01,  8.8379e-01]]),
       ),
)

In [10]:
b.a

tensor([[-0.8319, -0.7362, -0.4087],
        [ 0.3903, -1.5619, -0.4698]])

In [11]:
%timeit b.a

58 ns ± 0.0523 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1771,  2.0465,  2.6860],
               [ 0.9369, -1.5521, -0.3848]]),
    x: Batch(
           c: tensor([[-4.1987e-01,  1.0466e-03,  4.8218e-02, -1.9142e-01],
                      [-4.8661e-01, -1.4130e+00,  1.0928e+00, -1.6362e+00],
                      [ 1.5164e-01, -6.4227e-01,  2.5758e-01,  8.8379e-01]]),
       ),
)

In [13]:
%timeit b.a = new_value

523 ns ± 0.434 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

839 ns ± 3.93 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 24.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3d9c917c70>
├── 'a' --> tensor([[[-0.8319, -0.7362, -0.4087],
│                    [ 0.3903, -1.5619, -0.4698]],
│           
│                   [[-0.8319, -0.7362, -0.4087],
│                    [ 0.3903, -1.5619, -0.4698]],
│           
│                   [[-0.8319, -0.7362, -0.4087],
│                    [ 0.3903, -1.5619, -0.4698]],
│           
│                   [[-0.8319, -0.7362, -0.4087],
│                    [ 0.3903, -1.5619, -0.4698]],
│           
│                   [[-0.8319, -0.7362, -0.4087],
│                    [ 0.3903, -1.5619, -0.4698]],
│           
│                   [[-0.8319, -0.7362, -0.4087],
│                    [ 0.3903, -1.5619, -0.4698]],
│           
│                   [[-0.8319, -0.7362, -0.4087],
│                    [ 0.3903, -1.5619, -0.4698]],
│           
│                   [[-0.8319, -0.7362, -0.4087],
│                    [ 0.3903, -1.5619, -0.4698]]])
└── 'x' --> <FastTreeValue 0x7f3d9376bf40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.4 µs ± 18.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3d9c8b26a0>
├── 'a' --> tensor([[-0.8319, -0.7362, -0.4087],
│                   [ 0.3903, -1.5619, -0.4698],
│                   [-0.8319, -0.7362, -0.4087],
│                   [ 0.3903, -1.5619, -0.4698],
│                   [-0.8319, -0.7362, -0.4087],
│                   [ 0.3903, -1.5619, -0.4698],
│                   [-0.8319, -0.7362, -0.4087],
│                   [ 0.3903, -1.5619, -0.4698],
│                   [-0.8319, -0.7362, -0.4087],
│                   [ 0.3903, -1.5619, -0.4698],
│                   [-0.8319, -0.7362, -0.4087],
│                   [ 0.3903, -1.5619, -0.4698],
│                   [-0.8319, -0.7362, -0.4087],
│                   [ 0.3903, -1.5619, -0.4698],
│                   [-0.8319, -0.7362, -0.4087],
│                   [ 0.3903, -1.5619, -0.4698]])
└── 'x' --> <FastTreeValue 0x7f3d9483dbb0>
    └── 'c' --> tensor([[-4.1987e-01,  1.0466e-03,  4.8218e-02, -1.9142e-01],
                        [-4.8661e-01, -1.4130e+00,  1.0928e+00,

In [23]:
%timeit t_cat(trees)

31.1 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.5 µs ± 94.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.8319, -0.7362, -0.4087],
                [ 0.3903, -1.5619, -0.4698]],
       
               [[-0.8319, -0.7362, -0.4087],
                [ 0.3903, -1.5619, -0.4698]],
       
               [[-0.8319, -0.7362, -0.4087],
                [ 0.3903, -1.5619, -0.4698]],
       
               [[-0.8319, -0.7362, -0.4087],
                [ 0.3903, -1.5619, -0.4698]],
       
               [[-0.8319, -0.7362, -0.4087],
                [ 0.3903, -1.5619, -0.4698]],
       
               [[-0.8319, -0.7362, -0.4087],
                [ 0.3903, -1.5619, -0.4698]],
       
               [[-0.8319, -0.7362, -0.4087],
                [ 0.3903, -1.5619, -0.4698]],
       
               [[-0.8319, -0.7362, -0.4087],
                [ 0.3903, -1.5619, -0.4698]]]),
    x: Batch(
           c: tensor([[[-4.1987e-01,  1.0466e-03,  4.8218e-02, -1.9142e-01],
                       [-4.8661e-01, -1.4130e+00,  1.0928e+00, -1.6362e+00],
                       [ 1.5164e-01, -6

In [26]:
%timeit Batch.stack(batches)

78.9 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.8319, -0.7362, -0.4087],
               [ 0.3903, -1.5619, -0.4698],
               [-0.8319, -0.7362, -0.4087],
               [ 0.3903, -1.5619, -0.4698],
               [-0.8319, -0.7362, -0.4087],
               [ 0.3903, -1.5619, -0.4698],
               [-0.8319, -0.7362, -0.4087],
               [ 0.3903, -1.5619, -0.4698],
               [-0.8319, -0.7362, -0.4087],
               [ 0.3903, -1.5619, -0.4698],
               [-0.8319, -0.7362, -0.4087],
               [ 0.3903, -1.5619, -0.4698],
               [-0.8319, -0.7362, -0.4087],
               [ 0.3903, -1.5619, -0.4698],
               [-0.8319, -0.7362, -0.4087],
               [ 0.3903, -1.5619, -0.4698]]),
    x: Batch(
           c: tensor([[-4.1987e-01,  1.0466e-03,  4.8218e-02, -1.9142e-01],
                      [-4.8661e-01, -1.4130e+00,  1.0928e+00, -1.6362e+00],
                      [ 1.5164e-01, -6.4227e-01,  2.5758e-01,  8.8379e-01],
                      [-4.1987e-01,  1.0466e-

In [28]:
%timeit Batch.cat(batches)

150 µs ± 260 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

351 µs ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
